In [33]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
from itertools import product
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import pandas as pd
import random
import time

# 回测框架
import backtrader as bt

# 基础函数
import utilsJ

# 策略

In [34]:
class Strategy_comp_lite(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('p_stake', 100),

        # Indicator params
        ('bollinger_per', 20),
        ('bollinger_dev', 2),
        ('wave_period_buy', 7),
        ('wave_period_sell', 7),


        # ATR params
        ('using_atr', False),
        ('atr_period', 14),
        ('atr_percent', 1),
    )

    
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))

    
    def __init__(self):

        # Record ATR Initial
        self.atr_initial = self.broker.get_cash()
        self.inds = dict()
        self.profit_i = dict()
        self.data_tracker = None

        for d in self.datas:

            self.inds[d] = dict()
            self.profit_i[d._name] = 0

            # Keep references to lines in the data[0] dataseries
            self.inds[d]['dataclose'] = d.close
            self.inds[d]['datahigh'] = d.high
            self.inds[d]['datalow'] = d.low

            # Keep references to executed order & price
            self.inds[d]['buyprice'] = 0
            self.inds[d]['sellprice'] = 0
            self.inds[d]['order'] = None

            # Add indicators
            ## Bollinger Bands indicator
            self.inds[d]['bolling'] = bt.indicators.BollingerBands(d, 
                                                        period=self.params.bollinger_per, 
                                                        devfactor=self.params.bollinger_dev)
            self.inds[d]['bolling_buy'] = bt.Or(bt.And(self.inds[d]['dataclose'](0) > self.inds[d]['bolling'].top(0), 
                                                       self.inds[d]['dataclose'](-1) < self.inds[d]['bolling'].top(-1)), 
                                                bt.And(self.inds[d]['dataclose'](0) > self.inds[d]['bolling'].bot(0), 
                                                       self.inds[d]['dataclose'](-1) < self.inds[d]['bolling'].bot(-1)))
            self.inds[d]['bolling_sell'] = bt.Or(bt.And(self.inds[d]['dataclose'](0) < self.inds[d]['bolling'].top(0), 
                                                        self.inds[d]['dataclose'](-1) > self.inds[d]['bolling'].top(-1)), 
                                                 bt.And(self.inds[d]['dataclose'](0) < self.inds[d]['bolling'].bot(0), 
                                                        self.inds[d]['dataclose'](-1) > self.inds[d]['bolling'].bot(-1)))


            ## Wave indicator
            ### MovingAverageSimple indicator
            self.inds[d]['sma_buy'] = bt.indicators.SimpleMovingAverage(d, period=self.params.wave_period_buy)
            self.inds[d]['sma_sell'] = bt.indicators.SimpleMovingAverage(d, period=self.params.wave_period_sell)
            self.inds[d]['wave_buy'] = bt.And(self.inds[d]['sma_buy'](0) > self.inds[d]['sma_buy'](-1), 
                                              self.inds[d]['sma_buy'](-1) < self.inds[d]['sma_buy'](-2), 
                                              self.inds[d]['sma_buy'](-2) < self.inds[d]['sma_buy'](-3),
                                              self.inds[d]['sma_buy'](0) > self.inds[d]['bolling'].mid(0)),
                                             

            self.inds[d]['wave_sell'] = bt.And(self.inds[d]['sma_sell'](0) < self.inds[d]['sma_sell'](-1), 
                                               self.inds[d]['sma_sell'](-1) > self.inds[d]['sma_sell'](-2), 
                                               self.inds[d]['sma_sell'](-2) > self.inds[d]['sma_sell'](-3))
        
            ## ATR indicator
            self.inds[d]['tr'] = bt.indicators.Max((self.inds[d]['datahigh'](0) - self.inds[d]['datalow'](0)), 
                                                    abs(self.inds[d]['dataclose'](-1) - self.inds[d]['datahigh'](0)),
                                                    abs(self.inds[d]['dataclose'](-1) - self.inds[d]['datalow'](0)))
            self.inds[d]['atr'] = bt.indicators.SimpleMovingAverage(self.inds[d]['tr'], period=self.params.atr_period)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                if self.inds[self.data_tracker]['wave_buy']:
                    self.inds[self.data_tracker]['buyprice']
                self.log('买单执行, 代码：%s, 价格:%.2f, 股数:%i, 持有现金:%i' %
                         (order.info['name'],
                          order.executed.price,
                          order.executed.size,
                          self.broker.get_cash()))
                self.profit_i[order.info['name']] -= order.executed.price*order.executed.size

            else:  # Sell
                self.inds[self.data_tracker]['sellprice'] = order.executed.price
                self.log('卖单执行, 代码：%s, 价格:%.2f, 股数:%i, 持有现金:%i' %
                        (order.info['name'],
                         order.executed.price,
                         -order.executed.size,
                         self.broker.get_cash()))
                self.profit_i[order.info['name']] -= order.executed.price*order.executed.size

        elif order.status in [order.Canceled]:
            self.log('订单取消：被撤销')

        elif order.status in [order.Margin]:
            self.log('订单取消：资金不足，代码：%s，持有资金:%i' %
                        (order.info['name'],
                         self.broker.get_cash()))

        elif order.status in [order.Rejected]:
            self.log('订单取消：被拒绝')

        # Write down: no pending order
        #self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('交易收益 代码：%s, 毛利润 %.2f, 净利润 %.2f' %
                 (trade.history[0]['event']['order'].info['name'], 
                  trade.pnl, trade.pnlcomm))
    

    def next(self):

        buy_list = []
        sell_list = []

        for d in self.datas:
        # Check if an order is pending ... if yes, we cannot send a 2nd one
            if self.inds[d]['order']:
                return

            buy_indicator = self.inds[d]['wave_buy'] and self.inds[d]['dataclose'][0] > self.inds[d]['bolling'].mid[0]
            sell_indicator = self.inds[d]['wave_sell'] or self.inds[d]['bolling_sell']

            if sell_indicator and self.getposition(d).size > 0:
                sell_list.append((d, -1))

            elif buy_indicator and self.getposition(d).size == 0:
                if self.params.using_atr:
                    opt_pos = np.floor(self.atr_initial * self.params.atr_percent / self.params.p_stake / self.inds[d]['atr'][0])
                    if opt_pos*self.params.p_stake > 0 and self.getposition(d).size < opt_pos*self.params.p_stake:
                        buy_list.append((d, 
                                        opt_pos*self.params.p_stake-self.getposition(d).size, 
                                        (opt_pos*self.params.p_stake-self.getposition(d).size)*self.inds[d]['dataclose'][0]))
                else:
                    buy_list.append((d, 
                                     opt_pos*self.params.p_stake, 
                                     opt_pos*self.params.p_stake*self.inds[d]['dataclose'][0]))
        
        for s_order in sell_list:
            self.data_tracker = s_order[0]
            self.log('卖单创建, 代码: %s, 价格: %.2f, 股数:%i, 现有持仓: %i' % 
                        (s_order[0]._name, 
                        self.inds[s_order[0]]['dataclose'][0],
                        s_order[1] if s_order[1] != -1 else self.getposition(s_order[0]).size,
                        self.getposition(s_order[0]).size))
            self.sell(data=s_order[0], 
                        size=s_order[1] if s_order[1] != -1 else self.getposition(s_order[0]).size, 
                        name=s_order[0]._name)
        
        #buy_list = sorted(buy_list, key=lambda x: x[2], reverse=False)
        random.shuffle(buy_list)
        for b_order in buy_list:
            self.data_tracker = b_order[0]
            self.log('买单创建, 代码: %s, 价格: %.2f, 股数: %i, 现有持仓: %i' % 
                        (b_order[0]._name, 
                        self.inds[b_order[0]]['dataclose'][0], 
                        b_order[1],
                        self.getposition(b_order[0]).size))
            self.buy(data=b_order[0], size=b_order[1], name=b_order[0]._name)


    def stop(self):
        #for d in self.datas:
        #    if self.getposition(d).size > 0:
        #        self.profit_i[d._name] += self.getposition(d).size * d.close[0]
        #self.log('Ending Value:%.2f.' %
        #        (self.cerebro.broker.getvalue()), doprint=False)
        #print(sorted(self.profit_i.items(), key=lambda x: x[1]))
        #print(sum([x[1] for x in self.profit_i.items()]))
        return

# 全市场回测

### 数据下载

In [8]:
s_date = datetime.date(2020, 12, 31) - datetime.timedelta(days = 365 * 2)
e_date = datetime.date(2020, 12, 31)

#s_date = datetime.datetime.now() - datetime.timedelta(days = 365)
#e_date = datetime.datetime.now()

stock_index = '399300.SZ'

# Download Data
utilsJ.index_to_csv_tushare('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6',
                            stock_index, 0.2, s_date, e_date, fpath='.\\Data\\2019\\')

### 循环回测

In [32]:
profit_dict = {}

# 904ff4752522814dca00e032a709fdfc26d8744913500ef204e02157
# 74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6

def getdata_mul(s_code, start=datetime.date(2020, 12, 31)-datetime.timedelta(days = 385),
                end = datetime.date(2020, 12, 31)):
    global cerebro
    df = pd.read_csv('.\\Data\\2019\\'+s_code+'.csv', 
                     converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
    data = bt.feeds.PandasData(dataname=df, fromdate=s_date, todate=e_date)
    cerebro.adddata(data, name=s_code)
    return

s_date = datetime.date(2020, 12, 31) - datetime.timedelta(days = 385)
e_date = datetime.date(2020, 12, 31)



stock_index = '399300.SZ'
pro = ts.pro_api('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6')
index_list = np.unique(pro.index_weight(index_code=stock_index,
                                        start_date=s_date.strftime('%Y%m%d'),
                                        end_date=e_date.strftime('%Y%m%d')).con_code).tolist()


if __name__ == '__main__':
    for i in range(10):
        random.seed(10086)
        # Create a cerebro entity
        cerebro = bt.Cerebro(tradehistory=True)

        # Add a strategy
        strats = cerebro.addstrategy(Strategy_comp_lite, printlog=False, using_atr=True, atr_percent=0.01)

        t_list = [utilsJ.MyThread(getdata_mul, (s_code, s_date, e_date)) for s_code in index_list]
        [t.start() for t in t_list]
        [t.join() for t in t_list]
        #for s_code in index_list:
        #    df = pd.read_csv('.\\Data\\2020\\'+s_code+'.csv', 
        #                     converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
        #    data = bt.feeds.PandasData(dataname=df, fromdate=s_date, todate=e_date)

            # Add the index Data Feed to Cerebo
        #    cerebro.adddata(data, name=s_code)
        
        # Set cash inside the strategy
        cerebro.broker = bt.brokers.BackBroker(coc=True)   
        cerebro.broker.setcash(10000000)

        # Set commission
        #cerebro.broker.setcommission()

        # Print out the starting conditions
        start_value = cerebro.broker.getvalue()
        #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

        # Run over everything
        cerebro.run(maxcpus=8)

        # Print out the final result
        final_value = cerebro.broker.getvalue()
        #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
        #print('(%s, %s) Net Profit: %.2f%%' % (s[0], s[1], (final_value - start_value) / start_value * 100))
        print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))


Net Profit: 31.50%
Net Profit: 22.06%


KeyboardInterrupt: 

In [21]:
random.seed(10086)

s_date = datetime.date(2020, 12, 31) - datetime.timedelta(days = 385)
e_date = datetime.date(2020, 12, 31)

stock_index = '399300.SZ'
#pro = ts.pro_api('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6')
#index_list = np.unique(pro.index_weight(index_code=stock_index,
#                                        start_date=s_date.strftime('%Y%m%d'),
#                                        end_date=e_date.strftime('%Y%m%d')).con_code).tolist()
cerebro = bt.Cerebro(tradehistory=True)

# Add a strategy
strats = cerebro.addstrategy(Strategy_comp_lite, printlog=True, using_atr=True, atr_percent=0.01) 

for s_code in index_list:
    df = pd.read_csv('.\\Data\\2019\\' + s_code + '.csv')
    df.index=pd.to_datetime(df.trade_date)
    data = bt.feeds.PandasData(dataname=df,fromdate=s_date,todate=e_date)

    # Add the index Data Feed to Cerebo
    cerebro.adddata(data, name=s_code)

# Set cash inside the strategy
cerebro.broker = bt.brokers.BackBroker(coc=True)
cerebro.broker.setcash(10000000)

# Set commission
#cerebro.broker.setcommission()

# Print out the starting conditions
start_value = cerebro.broker.getvalue()
#print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run(maxcpus=4)

# Print out the final result
final_value = cerebro.broker.getvalue()
#print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))
#profit.append((final_value - start_value) / start_value * 100)

2020-07-27: 买单创建, 代码: 000157.SZ, 价格: 8.22, 股数: 7200, 现有持仓: 0
2020-07-27: 买单创建, 代码: 603288.SH, 价格: 144.99, 股数: 500, 现有持仓: 0
2020-07-27: 买单创建, 代码: 601899.SH, 价格: 6.02, 股数: 8200, 现有持仓: 0
2020-07-27: 买单创建, 代码: 300142.SZ, 价格: 68.08, 股数: 600, 现有持仓: 0
2020-07-27: 买单创建, 代码: 600547.SH, 价格: 43.98, 股数: 1700, 现有持仓: 0
2020-07-27: 买单创建, 代码: 000768.SZ, 价格: 22.38, 股数: 1700, 现有持仓: 0
2020-07-27: 买单创建, 代码: 600196.SH, 价格: 52.93, 股数: 900, 现有持仓: 0
2020-07-27: 买单创建, 代码: 601989.SH, 价格: 4.90, 股数: 9300, 现有持仓: 0
2020-07-27: 买单创建, 代码: 600989.SH, 价格: 10.12, 股数: 8900, 现有持仓: 0
2020-07-27: 买单创建, 代码: 600219.SH, 价格: 2.34, 股数: 24400, 现有持仓: 0
2020-07-27: 买单创建, 代码: 000703.SZ, 价格: 10.11, 股数: 6400, 现有持仓: 0
2020-07-27: 买单创建, 代码: 601600.SH, 价格: 3.26, 股数: 15900, 现有持仓: 0
2020-07-27: 买单创建, 代码: 601555.SH, 价格: 10.45, 股数: 4200, 现有持仓: 0
2020-07-27: 买单创建, 代码: 000858.SZ, 价格: 204.10, 股数: 200, 现有持仓: 0
2020-07-27: 买单创建, 代码: 300015.SZ, 价格: 46.00, 股数: 1500, 现有持仓: 0
2020-07-27: 买单创建, 代码: 603259.SH, 价格: 106.78, 股数: 400, 现有持仓: 0
2020-07-27: 买

### 循环调参

In [37]:
profit_dict = {}

# 904ff4752522814dca00e032a709fdfc26d8744913500ef204e02157
# 74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6

def getdata_mul(s_code, start=datetime.date(2020, 12, 31)-datetime.timedelta(days = 385),
                end = datetime.date(2020, 12, 31)):
    global cerebro
    df = pd.read_csv('.\\Data\\2019\\'+s_code+'.csv', 
                     converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
    data = bt.feeds.PandasData(dataname=df, fromdate=s_date, todate=e_date)
    cerebro.adddata(data, name=s_code)
    return

s_date = datetime.date(2020, 12, 31) - datetime.timedelta(days = 385)
e_date = datetime.date(2020, 12, 31)



stock_index = '399300.SZ'
pro = ts.pro_api('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6')
index_list = np.unique(pro.index_weight(index_code=stock_index,
                                        start_date=s_date.strftime('%Y%m%d'),
                                        end_date=e_date.strftime('%Y%m%d')).con_code).tolist()


if __name__ == '__main__':
    for i in range(3,10):
        for j in range(3,10):
            profit = list()
            for _ in range(10):
                random.seed(10086)
                # Create a cerebro entity
                cerebro = bt.Cerebro(tradehistory=True)

                # Add a strategy
                strats = cerebro.addstrategy(Strategy_comp_lite, printlog=False, using_atr=True, atr_percent=0.01, wave_period_buy=i, wave_period_sell=j)

                t_list = [utilsJ.MyThread(getdata_mul, (s_code, s_date, e_date)) for s_code in index_list]
                [t.start() for t in t_list]
                [t.join() for t in t_list]
                #for s_code in index_list:
                #    df = pd.read_csv('.\\Data\\2020\\'+s_code+'.csv', 
                #                     converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
                #    data = bt.feeds.PandasData(dataname=df, fromdate=s_date, todate=e_date)

                    # Add the index Data Feed to Cerebo
                #    cerebro.adddata(data, name=s_code)
                
                # Set cash inside the strategy
                cerebro.broker = bt.brokers.BackBroker(coc=True)   
                cerebro.broker.setcash(10000000)

                # Set commission
                #cerebro.broker.setcommission()

                # Print out the starting conditions
                start_value = cerebro.broker.getvalue()
                #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

                # Run over everything
                cerebro.run(maxcpus=16)

                # Print out the final result
                final_value = cerebro.broker.getvalue()
                #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
                #print('(%s, %s) Net Profit: %.2f%%' % (s[0], s[1], (final_value - start_value) / start_value * 100))
                profit.append((final_value - start_value) / start_value * 100)
            print((i, j, max(profit), min(profit), np.mean(profit)))

(3, 3, 14.40239, -15.90477, 1.009067000000006)
(3, 4, 16.30648000000002, -10.61242, 2.813963000000002)
(3, 5, 33.86727000000002, -10.802540000000018, 11.722723000000002)
(3, 6, 26.473160000000018, -6.492979999999999, 13.769436000000002)


KeyboardInterrupt: 